In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import scipy as sp
import scipy.stats
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import re
res_digit = r'[0-9]'

# fourier transform
from scipy.fft import fft, ifft

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.manifold import TSNE



In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

from predicament.utils.config import FEATURED_BASE_PATH

In [4]:
from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition

from predicament.data.features import MAXIMAL_FEATURE_GROUP
from predicament.data.features import STATS_FEATURE_GROUP
from predicament.data.features import INFO_FEATURE_GROUP
from predicament.data.features import FREQ_FEATURE_GROUP
from predicament.data.features import convert_timeseries_to_features
from prepare_evaluation_data import load_dataframe_and_config


from predicament.data.datasets import propose_balanced_subject_condition_counts
from predicament.data.datasets import subsample_proposed_subject_condition_counts
from predicament.data.datasets import get_subject_condition_counts

from predicament.evaluation.results import output_model_best_from_results
from predicament.evaluation.results import save_results_df_to_file


In [31]:
from predicament.evaluation.results import test
from predicament.evaluation.results import save_results_df_to_file


,model,held out,feature set,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,...,split9_train_score,split10_train_score,split11_train_score,mean_train_score,std_train_score,param_activation,param_alpha,param_hidden_layer_sizes,param_learning_rate,param_solver
0,Random Forest,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",0.878352,0.027888,0.003233,0.000415,10,{'n_estimators': 10},0.266507,...,0.965878,0.965555,0.968225,0.965971,0.001712,NaN,NaN,NaN,NaN,NaN
1,Random Forest,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",1.804766,0.065974,0.005504,0.000771,20,{'n_estimators': 20},0.273709,...,0.980326,0.982197,0.982521,0.981154,0.000878,NaN,NaN,NaN,NaN,NaN
2,Random Forest,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",4.647198,0.199652,0.013108,0.002702,50,{'n_estimators': 50},0.281513,...,0.988196,0.990066,0.989013,0.988590,0.000701,NaN,NaN,NaN,NaN,NaN
3,Random Forest,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",9.671321,0.739652,0.024006,0.003322,100,{'n_estimators': 100},0.280312,...,0.990582,0.992066,0.991385,0.990724,0.000733,NaN,NaN,NaN,NaN,NaN
0,Gradient Boosting,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",17.824586,0.337792,0.001995,0.000280,10,{'n_estimators': 10},0.207083,...,0.542927,0.530478,0.540046,0.541370,0.009937,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,MLP,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",7.068870,1.437706,0.002646,0.000426,NaN,"{'activation': 'relu', 'alpha': 0.05, 'hidden_...",0.226891,...,0.301168,0.207379,0.229852,0.271346,0.034374,relu,0.05,"(50,)",adaptive,adam
92,MLP,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",3.012101,0.527593,0.003294,0.000508,NaN,"{'activation': 'relu', 'alpha': 0.05, 'hidden_...",0.169868,...,0.215249,0.215249,0.208378,0.208251,0.005259,relu,0.05,"(100,)",constant,sgd
93,MLP,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",8.229035,2.249828,0.003388,0.000507,NaN,"{'activation': 'relu', 'alpha': 0.05, 'hidden_...",0.186675,...,0.280978,0.245630,0.299894,0.262453,0.027812,relu,0.05,"(100,)",constant,adam
94,MLP,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",10.277087,0.514650,0.003371,0.000524,NaN,"{'activation': 'relu', 'alpha': 0.05, 'hidden_...",0.181873,...,0.215249,0.215249,0.208752,0.208299,0.005210,relu,0.05,"(100,)",adaptive,sgd


## Load features

Before running this, you will need to generate featured data. See README file for details. For the variable `subdir` below replace this with the subdirectory name of the featured data.

In [5]:
subdir = '20231129210920'
featured_data_dir = os.path.join(FEATURED_BASE_PATH,subdir)

featured_df, featured_config = load_dataframe_and_config(
    featured_data_dir, 'featured.csv')
n_channels = int(featured_config['LOAD']['n_channels'])
channels = json.loads(featured_config['LOAD']['channels'].replace("'",'"'))
participant_list = json.loads(featured_config['LOAD']['participant_list'].replace("'",'"'))
Fs = int(featured_config['LOAD']['sample_rate'])
window_size = int(featured_config['LOAD']['window_size'])
time = window_size/Fs
print(f"Fs: {Fs}, n_samples = {window_size}, time: {time}s, n_channels: {n_channels}")


Fs: 250, n_samples = 1024, time: 4.096s, n_channels: 5


In [6]:
featured_df.columns

Index(['part_ID', 'condition', 'start time', 'Mean0', 'Mean1', 'Mean2',
       'Mean3', 'Mean4', 'SD0', 'SD1', 'SD2', 'SD3', 'SD4', 'MAD0', 'MAD1',
       'MAD2', 'MAD3', 'MAD4', 'Max0', 'Max1', 'Max2', 'Max3', 'Max4', 'Min0',
       'Min1', 'Min2', 'Min3', 'Min4', 'Energy0', 'Energy1', 'Energy2',
       'Energy3', 'Energy4', 'IQR0', 'IQR1', 'IQR2', 'IQR3', 'IQR4',
       'Correlation0', 'Correlation1', 'Correlation2', 'Correlation3',
       'Correlation4', 'Correlation5', 'Correlation6', 'Correlation7',
       'Correlation8', 'Correlation9', 'arCoeff0', 'arCoeff1', 'arCoeff2',
       'arCoeff3', 'arCoeff4', 'arCoeff5', 'arCoeff6', 'arCoeff7', 'arCoeff8',
       'arCoeff9', 'arCoeff10', 'arCoeff11', 'arCoeff12', 'arCoeff13',
       'arCoeff14', 'arCoeff15', 'arCoeff16', 'arCoeff17', 'arCoeff18',
       'arCoeff19', 'LyapunovExponent0', 'LyapunovExponent1',
       'LyapunovExponent2', 'LyapunovExponent3', 'LyapunovExponent4',
       'MaxFreqInd0', 'MaxFreqInd1', 'MaxFreqInd2', 'MaxFreqI

In [7]:
# # you can choose a subset of the feature types to use
# features_to_use = set(
#     ['Mean', 'SD', 'MAD', 'Max', 'Min',
#       'Energy', 'IQR',
#      'arCoeff', 'Correlation', 'MaxFreqInd', 'MeanFreq', 'FreqSkewness',
#     'FreqKurtosis'
#     ])
# # this constructs a list of the featured data columns based on the above feature types
# columns_to_use = [ col for col in featured_df.columns if re.sub(res_digit, '', col) in features_to_use ]
# print(f"columns_to_use = {columns_to_use}")

# designmtx = featured_df[columns_to_use].values 
# condition_data = featured_df['condition'].values.astype(int)
# subject_data_names = featured_df['part_ID']

# design2d = TSNE(n_components=2, init='random', perplexity=3).fit_transform(designmtx)
# print(f"design2d.shape = {design2d.shape}")

columns_to_use = ['Mean0', 'Mean1', 'Mean2', 'Mean3', 'Mean4', 'SD0', 'SD1', 'SD2', 'SD3', 'SD4', 'MAD0', 'MAD1', 'MAD2', 'MAD3', 'MAD4', 'Max0', 'Max1', 'Max2', 'Max3', 'Max4', 'Min0', 'Min1', 'Min2', 'Min3', 'Min4', 'Energy0', 'Energy1', 'Energy2', 'Energy3', 'Energy4', 'IQR0', 'IQR1', 'IQR2', 'IQR3', 'IQR4', 'Correlation0', 'Correlation1', 'Correlation2', 'Correlation3', 'Correlation4', 'Correlation5', 'Correlation6', 'Correlation7', 'Correlation8', 'Correlation9', 'arCoeff0', 'arCoeff1', 'arCoeff2', 'arCoeff3', 'arCoeff4', 'arCoeff5', 'arCoeff6', 'arCoeff7', 'arCoeff8', 'arCoeff9', 'arCoeff10', 'arCoeff11', 'arCoeff12', 'arCoeff13', 'arCoeff14', 'arCoeff15', 'arCoeff16', 'arCoeff17', 'arCoeff18', 'arCoeff19', 'MaxFreqInd0', 'MaxFreqInd1', 'MaxFreqInd2', 'MaxFreqInd3', 'MaxFreqInd4', 'MeanFreq0', 'MeanFreq1', 'MeanFreq2', 'MeanFreq3', 'MeanFreq4', 'FreqSkewness0', 'FreqSkewness1', 'FreqSkewness2', 'FreqSkewness3', 'FreqSkewness4', 'FreqKurtosis0', 'FreqKurtosis1', 'FreqKurtosis2', '

In [8]:
# balancing the data
proposed_subject_condition_counts = propose_balanced_subject_condition_counts(
    featured_df)
print(f"proposed_subject_condition_counts = {proposed_subject_condition_counts}")
proposed_condition_counts = np.sum(proposed_subject_condition_counts, axis=0)
print(f"proposed_condition_counts = {proposed_condition_counts}")
proposed_class_balance = proposed_condition_counts/np.sum(proposed_condition_counts)
print(f"proposed_class_balance = {proposed_class_balance}")
print(f"balancing featured data...")
featured_df = subsample_proposed_subject_condition_counts(
    featured_df, proposed_subject_condition_counts)
# post-check
subject_condition_counts = get_subject_condition_counts(featured_df)
print(f"after balancing: subject_condition_counts = {subject_condition_counts}")

proposed_subject_condition_counts = [[304 322 461 283 296]
 [304 322 461 283 296]
 [304 110 344 283 296]
 [304 227 461 283 296]
 [304 322 344 227 296]
 [  0 110   0 283 296]
 [304 322   0 283 296]
 [304 322 461 283  86]
 [304 322   0 283 296]
 [304 322   0 283 296]
 [304 322 344 283 296]
 [304 322 461 283 296]]
proposed_condition_counts = [3344 3345 3337 3340 3342]
proposed_class_balance = [0.20014364 0.2002035  0.19972468 0.19990424 0.20002394]


## Classification

In [ ]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer


In [17]:
# models = {
#     "Random Forest": RandomForestClassifier(
#         min_samples_leaf=5, random_state=0
#     ),
#     "Gradient Boosting": GradientBoostingClassifier(
#         max_leaf_nodes=15, random_state=0
#     ),
#     "MLP":  MLPClassifier(max_iter=100)
# }
# param_grids = {
#     "Random Forest": {"n_estimators": [10, 20, 50, 100]},
#     "Gradient Boosting": {"n_estimators": [10, 20, 50, 100]},
#     "MLP": {
#         'hidden_layer_sizes': [(10,),(20,),(50,),(100,)],
#         'activation': ['relu'],
#         'solver': ['sgd', 'adam'],
#         'alpha': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05],
#         'learning_rate': ['constant','adaptive'],
#     }
# }

# you can choose a subset of the feature types to use
features_to_use = set(
    ['Mean', 'SD', 'MAD', 'Max', 'Min',
      'Energy', 'IQR',
     'arCoeff', 'Correlation', 'MaxFreqInd', 'MeanFreq', 'FreqSkewness',
    'FreqKurtosis'
    ])
# this constructs a list of the featured data columns based on the above feature types
columns_to_use = [ col for col in featured_df.columns if re.sub(res_digit, '', col) in features_to_use ]
print(f"columns_to_use = {columns_to_use}")

designmtx = featured_df[columns_to_use].to_numpy()

condition_data = featured_df['condition'].values.astype(int)

columns_to_use = ['Mean0', 'Mean1', 'Mean2', 'Mean3', 'Mean4', 'SD0', 'SD1', 'SD2', 'SD3', 'SD4', 'MAD0', 'MAD1', 'MAD2', 'MAD3', 'MAD4', 'Max0', 'Max1', 'Max2', 'Max3', 'Max4', 'Min0', 'Min1', 'Min2', 'Min3', 'Min4', 'Energy0', 'Energy1', 'Energy2', 'Energy3', 'Energy4', 'IQR0', 'IQR1', 'IQR2', 'IQR3', 'IQR4', 'Correlation0', 'Correlation1', 'Correlation2', 'Correlation3', 'Correlation4', 'Correlation5', 'Correlation6', 'Correlation7', 'Correlation8', 'Correlation9', 'arCoeff0', 'arCoeff1', 'arCoeff2', 'arCoeff3', 'arCoeff4', 'arCoeff5', 'arCoeff6', 'arCoeff7', 'arCoeff8', 'arCoeff9', 'arCoeff10', 'arCoeff11', 'arCoeff12', 'arCoeff13', 'arCoeff14', 'arCoeff15', 'arCoeff16', 'arCoeff17', 'arCoeff18', 'arCoeff19', 'MaxFreqInd0', 'MaxFreqInd1', 'MaxFreqInd2', 'MaxFreqInd3', 'MaxFreqInd4', 'MeanFreq0', 'MeanFreq1', 'MeanFreq2', 'MeanFreq3', 'MeanFreq4', 'FreqSkewness0', 'FreqSkewness1', 'FreqSkewness2', 'FreqSkewness3', 'FreqSkewness4', 'FreqKurtosis0', 'FreqKurtosis1', 'FreqKurtosis2', '

## Hold one group out

In [18]:
# standard cross-validation
# Match all digits in the string and replace them with an empty string
# new_string = re.sub(pattern, '', string1)
subjects = np.unique(featured_df['part_ID'])
n_subjects = len(subjects)
groups = np.empty(len(featured_df), dtype=int)
for s, sub in enumerate(subjects):
    groups[featured_df['part_ID']==sub] = s
# cross validation splits    
group_kfold = GroupKFold(n_splits=n_subjects)


In [32]:
save_results_df_to_file(results_df, 'balanced_eeg')

saving to ../data/results/balanced_eeg_2023-12-12T08:43:49.csv


In [33]:
output_model_best_from_results(results_df)

Gradient Boosting: max_test_score= 0.22389534372491024, max_std_test_score= 0.06334182766052263
best params: {'n_estimators': 100}

MLP: max_test_score= 0.2656577634163771, max_std_test_score= 0.10001973114790763
best params: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (20,), 'learning_rate': 'constant', 'solver': 'adam'}

Random Forest: max_test_score= 0.21415626535025437, max_std_test_score= 0.07623024243583791
best params: {'n_estimators': 100}



## Random Search

In [36]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False] 
# Create the random grid
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap}

In [37]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
estimator = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
param_search = RandomizedSearchCV(
    estimator = estimator,
    param_distributions = random_grid,
    n_iter = 100,
    cv=group_kfold,
    verbose=2,
    random_state=42,
    n_jobs = -1)


# Fit the random search model
param_search.fit(X=designmtx, y=condition_data, groups=groups)
result_df = pd.DataFrame(param_search.cv_results_)
result_df.insert(0, 'model', name)
result_df.insert(1, 'held out', 'subject')
result_df.insert(2, 'feature set', str(features_to_use))
display(result_df)
save_results_df_to_file(result_df, 'random_search_random_forest')

Fitting 12 folds for each of 100 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 29.7min


[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=  56.0s
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total= 1.1min
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total= 1.3min
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True, total= 4.3min
[CV] n_estimators=2000, min_sample

/usr/lib/python3/dist-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=  58.7s
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total= 1.1min
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=400, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total= 1.3min
[CV] n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=2000, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True, total= 4.4min
[CV] n_estimators=2000, min_sample

...

[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=2000, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=False, total=12.5min
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=2000, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=False, total=12.9min
[CV] n_estimators=1600, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=1600, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=10, bootstrap=True, total= 4.8min
[CV] n_estimators=1600, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=1600, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=10, bootstrap=True, total= 4.9min
[CV] n_estimators=1600, 

[CV]  n_estimators=800, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=30, bootstrap=False, total= 3.5min
[CV] n_estimators=800, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=30, bootstrap=False 
[CV]  n_estimators=800, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=30, bootstrap=False, total= 3.5min
[CV] n_estimators=800, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=30, bootstrap=False 
[CV]  n_estimators=800, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=30, bootstrap=False, total= 3.6min
[CV] n_estimators=1800, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False, total= 7.6min
[CV] n_estimators=1800, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False 
[CV]  n_estimators=1

[CV]  n_estimators=800, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=30, bootstrap=False, total= 3.6min
[CV] n_estimators=1800, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False, total= 7.3min
[CV] n_estimators=1800, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False, total= 7.7min
[CV] n_estimators=1800, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False, total= 8.2min
[CV] n_estimators=600, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=70, bootstrap=False 
[CV]  n_estimators=60

[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 574.2min


KeyboardInterrupt: 

## Bayesian Optimisation
May be quicker and more effective than random search

In [38]:
# Bayesian optimisation



estimator = MLPClassifier(max_iter=100)
search_spaces =      {
        'hidden_layer_sizes': Categorical([ (n,) for n in range(10,200,10)]),
        'activation': Categorical(['tanh', 'relu']),
        'solver': Categorical(['sgd', 'adam']),
        'alpha': Real(1e-6, 1e+1, prior='log-uniform'),
        'learning_rate': Categorical(['constant','adaptive']),
     }

# log-uniform: understand as search over p = exp(x) by varying x
param_search = BayesSearchCV(
    estimator, search_spaces,
    cv=group_kfold, verbose=2, random_state=42, n_iter=50)

# executes bayesian optimization
_ = param_search.fit(X=designmtx, y=condition_data, groups=groups)
result_df = pd.DataFrame(param_search.cv_results_)
result_df.insert(0, 'model', name)
result_df.insert(1, 'held out', 'subject')
result_df.insert(2, 'feature set', str(features_to_use))
display(result_df)

save_results_df_to_file(result_df, 'bayes_search')



Fitting 12 folds for each of 1 candidates, totalling 12 fits
[CV] activation=relu, alpha=0.1241881627783703, hidden_layer_sizes=180, learning_rate=adaptive, solver=sgd 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/lib/python3/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.7s remaining:    0.0s


[CV]  activation=relu, alpha=0.1241881627783703, hidden_layer_sizes=180, learning_rate=adaptive, solver=sgd, total=  15.7s
[CV] activation=relu, alpha=0.1241881627783703, hidden_layer_sizes=180, learning_rate=adaptive, solver=sgd 


/usr/lib/python3/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV]  activation=relu, alpha=0.1241881627783703, hidden_layer_sizes=180, learning_rate=adaptive, solver=sgd, total=  17.4s
[CV] activation=relu, alpha=0.1241881627783703, hidden_layer_sizes=180, learning_rate=adaptive, solver=sgd 


...

/usr/lib/python3/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV]  activation=relu, alpha=0.1241881627783703, hidden_layer_sizes=180, learning_rate=adaptive, solver=sgd, total=  18.4s
[CV] activation=relu, alpha=0.1241881627783703, hidden_layer_sizes=180, learning_rate=adaptive, solver=sgd 


/usr/lib/python3/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  3.5min finished


[CV]  activation=relu, alpha=0.1241881627783703, hidden_layer_sizes=180, learning_rate=adaptive, solver=sgd, total=  18.8s


ValueError: Not all points are within the bounds of the space.